In [ ]:
!pip install -U trl transformers accelerate git+https://github.com/huggingface/peft.git


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-5ar0xwko
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-5ar0xwko
  Resolved https://github.com/huggingface/peft.git to commit 9d8287f3e32be7b22bd77fe23b96054cff291642
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.2 MB/s e

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-20yy2f0x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-20yy2f0x
  Resolved https://github.com/huggingface/transformers.git to commit cf32c941350cb296e4c2c9e26a9274291d515e90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=7931218 sha256=532bdb6054386afaf1e8000301fd11e808269cf4fd21a56f61d4280eec0fd875
  Stored in directory: /tmp/pip-ephem-wheel-cache-mf4mmql8/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0
  Cloning https://github.com

In [ ]:
!pip install tensorflow


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from torch.utils.data import DataLoader, TensorDataset, random_split

from transformers import TrainingArguments
from transformers import Trainer
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam


In [ ]:
df_train=pd.read_csv("/content/drive/MyDrive/Software Project 2/train_data_chatbot.csv")

In [ ]:
df_train.head(5)

,short_question,short_answer,tags,label
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,['rash' 'antibiotic'],1.0
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,['hepatitis b'],1.0
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,['celiac disease'],1.0
3,can i transmit genital warts seventeen years a...,famotidine pepcid products is in a drug class ...,['wart'],-1.0
4,is all vitamin d the same,hi this means you do not have hepatitis b and ...,['vitamin d'],-1.0


In [ ]:
df_test=pd.read_csv("/content/drive/MyDrive/Software Project 2/test-data-small.csv")


In [ ]:
df_test.head(5)

,short_question,short_answer
0,is chalazion contagious my 2 daughters both ha...,no do not worry about it i agree with her doct...
1,what are the best exercises for building and m...,according to the national osteoporosis foundat...
2,when should i call the doctor regarding nausea...,some people are more prone to poor wound heali...
3,i recently got a rash on one side of my face c...,i would go back to my md and tell him to take ...
4,abdominal pain…is it a urinary tract infection,get a ultrasound done to rule out any other pa...


In [ ]:
columns_to_delete = ['tags', 'label']
df_train = df_train.drop(columns=columns_to_delete)




In [ ]:
df_train.head(2)

,short_question,short_answer
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...


In [ ]:
df_train = df_train.iloc[20000:]
df_test = df_test.iloc[5000:]

In [ ]:
df_train.shape, df_test.shape


((27603, 2), (1901, 2))

# Loading BERT tokenizer and encode the data

In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(df_train['short_question'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(df_test['short_question'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')


In [ ]:
train_labels = df_train['short_answer'].values
test_labels = df_test['short_answer'].values

In [ ]:
# Loading pre-trained BERT model for sequence classification


In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)


In [ ]:
model.compile(optimizer=Adam(lr=2e-5), loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
history = model.fit(
    x=train_encodings,
    y=train_labels,
    epochs=3,  # You may need to adjust the number of epochs based on your dataset
    batch_size=32,
    validation_split=0.1  # You can adjust the validation split
)


Epoch 1/3


ValueError: ignored